In [1]:
import csv
import pickle
import numpy as np
from sklearn.model_selection import train_test_split
import pandas as pd
import math

FILE_NAME = 'year2010_movie'
RATING_FILE = 'movie_year_2011_2016.csv'

In [2]:
def sigmoid(x):
  return 1 / (1 + math.exp(-x))

In [3]:
# load movie info matrix
index = 0
title_dict = {}
rating_dict = {}
director_dict = {}
cast_dict = {}
country_dict = {}
genre_dict = {}
year_dict = {}
lmid_2_id = {}
infor_dict = {}
with open('../data/'+FILE_NAME+'/movie_info.csv',encoding = "ISO-8859-1") as f:
    f_csv = csv.reader(f)
    headers = next(f_csv)
    for row in f_csv:
        infor_dict[row[0]] = row    #### row[0] is the given id in movie_info.csv


# every dictionary's id is the given movie id
for key in infor_dict:
    row = infor_dict[key]
    title_dict[key] = [row[4]]
    rating_dict[key] = [row[5]]
    director_dict[key] = row[6].split('<sp>')
    cast_dict[key] = row[7].split('<sp>')
    country_dict[key] = row[8].split('<sp>')
    genre_dict[key] = row[9].split('<sp>')
    year_dict[key] = [row[10]]
    # key: id in movielens 1m, value: id in id we use
    lmid_2_id[row[1]] = row[0]
        
        

## 规定，所有的user id为后面定义的user id， movie id用movie_info.csv里面的id

In [4]:
pickle.dump(lmid_2_id, open('../data/'+FILE_NAME+'/lmid_2_id.pkl', 'wb'))

In [5]:
len_dtcgum = [0, 0, 0, 0, 0, 0]

In [6]:
director_2_id = {}
id_2_director = {}
i = 0
for key in director_dict:
    for name in director_dict[key]:
        if name not in director_2_id:
            director_2_id[name] = i
            id_2_director[i] = name
            i += 1

In [7]:
max_director = 0
for key in id_2_director:
    if max_director < int(key):
        max_director = int(key)
len_dtcgum[0] = max_director + 1
max_director + 1

3715

In [8]:
# only calculate the first three casts
cast_2_id = {}
id_2_cast = {}
i = 0
for key in cast_dict:
    for name in cast_dict[key][0:3]:
        if name not in cast_2_id:
            cast_2_id[name] = i
            id_2_cast[i] = name
            i += 1

In [9]:
max_cast = 0
for key in id_2_cast:
    if max_cast < int(key):
        max_cast = int(key)
len_dtcgum[2] = max_cast + 1
max_cast + 1

8065

In [10]:
genre_2_id = {}
id_2_genre = {}
i = 0
for key in genre_dict:
    for name in genre_dict[key]:
        if name not in genre_2_id:
            genre_2_id[name] = i
            id_2_genre[i] = name
            i += 1

In [11]:
max_genre = 0
for key in id_2_genre:
    if max_genre < int(key):
        max_genre = int(key)
len_dtcgum[3] = max_genre + 1
max_genre + 1

26

In [12]:
#handle user id
mid_2_realid = {}
max_movie_id = 0
index = 0
for line in open('../data/rating_time/' + RATING_FILE, 'r'):
    index += 1
    #item = line.rstrip().split("::")
    item_info = line.rstrip().split(",")
    if item_info[0] != 'userid':
        max_movie_id = int(lmid_2_id[item_info[1]]) if int(lmid_2_id[item_info[1]]) > max_movie_id else max_movie_id
        if item_info[0] not in mid_2_realid:
            mid_2_realid[item_info[0]] = len(mid_2_realid)
            
        

In [13]:
index

252030

In [14]:
max_movie_id

3906

In [15]:
index = 0
for line in open('../data/rating_time/' + RATING_FILE, 'r'):
    item_info = line.rstrip().split(",")
    if item_info[0]  == '138419':
        index += 1
            
index

0

In [16]:
len_dtcgum[4] = len(mid_2_realid)
len_dtcgum[5] = max_movie_id

pickle.dump(len_dtcgum, open('../data/'+FILE_NAME+'/len_dtcgum.pkl', 'wb'))

In [17]:
index = 0
movie_rating = {}
train_temp = []
test_temp = []
for line in open('../data/rating_time/'+RATING_FILE, 'r'):
    #item = line.rstrip().split("::")
    item_info = line.rstrip().split(",")
    if item_info[0] != 'userid':
        item_info[0] = mid_2_realid[item_info[0]]
        item_info[1] = lmid_2_id[item_info[1]]
        train_temp.append(item_info)
    

data_tmp = train_temp
train, test_validation = train_test_split(data_tmp, test_size = 0.2)
test, validation = train_test_split(test_validation, test_size = 0.5)
#train, validation = train_test_split(train_temp, test_size = 0.25)
#test = test_temp

In [18]:
print(len(train), len(validation), len(test))

201623 25203 25203


In [19]:
def change_to_int(data):
    result = []
    for info in data:
        one_info = []
        for n in info[0:-2]:
            one_info.append(int(n))
        one_info.append(float(info[-2]))    
        
        result.append(one_info)
    return result

train_all = change_to_int(train)
test_all = change_to_int(test)
validation_all = change_to_int(validation)

In [20]:
pickle.dump(train_all, open('../data/'+FILE_NAME+'/train_all.pkl', 'wb'))
pickle.dump(test_all, open('../data/'+FILE_NAME+'/test_all.pkl', 'wb'))
pickle.dump(validation_all, open('../data/'+FILE_NAME+'/validation_all.pkl', 'wb'))

In [21]:
def gen_rating_data(df):
    user_rating = {}
    movie_rating = {}
    index = 0
    for item in df:
        try:
            #real_id is the 1m id
            user_id = item[0]
            movie_id = int(item[1])
            rating = float(item[2])
            user_rating[user_id] = [(movie_id, rating)] if user_id not in user_rating else (user_rating[user_id] + [(movie_id, rating)])
            movie_rating[movie_id] = [(user_id, rating)] if movie_id not in movie_rating else (movie_rating[movie_id] + [(user_id, rating)])
        except:
            index += 1
            pass
    
    print(index)
    return user_rating, movie_rating

In [22]:
user_rating, movie_rating = gen_rating_data(train)
user_rating_test, movie_rating_test = gen_rating_data(test)
user_rating_validation, movie_rating_validation = gen_rating_data(validation)

0
0
0


In [23]:
# data for training, the exact movie rating for each user
train_user = [int(inf[0]) for inf in train]
train_movie = [int(inf[1]) for inf in train]
train_rating = [float(inf[2]) for inf in train]

pickle.dump(train_user, open('../data/'+FILE_NAME+'/train_user.pkl', 'wb'))
pickle.dump(train_movie, open('../data/'+FILE_NAME+'/train_movie.pkl', 'wb'))
pickle.dump(train_rating, open('../data/'+FILE_NAME+'/train_rating.pkl', 'wb'))

In [24]:
# data for training, the exact movie rating for each user
valid_user = [int(inf[0]) for inf in validation]
valid_movie = [int(inf[1]) for inf in validation]
valid_rating = [float(inf[2]) for inf in validation]

pickle.dump(valid_user, open('../data/'+FILE_NAME+'/valid_user.pkl', 'wb'))
pickle.dump(valid_movie, open('../data/'+FILE_NAME+'/valid_movie.pkl', 'wb'))
pickle.dump(valid_rating, open('../data/'+FILE_NAME+'/valid_rating.pkl', 'wb'))

In [25]:
# data for testing, the exact movie rating for each user
test_user = [int(inf[0]) for inf in test]
test_movie = [int(inf[1]) for inf in test]
test_rating = [float(inf[2]) for inf in test]

pickle.dump(test_user, open('../data/'+FILE_NAME+'/test_user.pkl', 'wb'))
pickle.dump(test_movie, open('../data/'+FILE_NAME+'/test_movie.pkl', 'wb'))
pickle.dump(test_rating, open('../data/'+FILE_NAME+'/test_rating.pkl', 'wb'))

In [26]:
pickle.dump(user_rating, open('../data/'+FILE_NAME+'/user_rating.pkl', 'wb'))
pickle.dump(movie_rating, open('../data/'+FILE_NAME+'/movie_rating.pkl', 'wb'))
pickle.dump(user_rating_test, open('../data/'+FILE_NAME+'/user_rating_test.pkl', 'wb'))
pickle.dump(movie_rating_test, open('../data/'+FILE_NAME+'/movie_rating_test.pkl', 'wb'))
pickle.dump(user_rating_validation, open('../data/'+FILE_NAME+'/user_rating_validation.pkl', 'wb'))
pickle.dump(movie_rating_validation, open('../data/'+FILE_NAME+'/movie_rating_validation.pkl', 'wb'))

# if not change dataset, begin here

In [27]:
num_of_1 = 0
def core_fn(scores):
    global num_of_1
    """
    core function that wrap the score according to how 
    many number of scores contribute the score 
    Currently: the average score
    """
    # scores is the list of (proportion, rating)
    sum_p = 0.
    sum_r = 0.
    num = len(scores) - 1
    for tup in scores:
        prop = float(tup[0]) 
        r = float(tup[1])
        sum_r += prop * r
        #sum_r += r
        sum_p += prop
    
    if num == 1:
        num_of_1 += 1
    #return (sum_r/sum_p) * (0.3 + sigmoid(num+1))
    return (sum_r/sum_p), sum_p
    #result = sum_r/float(num) if num > 0 else 0
    #return result

def generate_proportion(info_list, is_cast):
    if is_cast:
        assert len(info_list) <= 3
        if len(info_list) == 3:
            return [(info_list[0], 1), (info_list[1], 0.8), (info_list[2], 0.6)]
        elif len(info_list) == 2:
            return [(info_list[0], 1), (info_list[1], 0.8)]   
        elif len(info_list) == 1:
            return [(info_list[0], 1)]
        else:
            return []
    else:
        result = []
        prop = 1./len(info_list) if len(info_list) > 0 else 0
        #prop = 1
        for _id in info_list:
            result.append((_id, prop))
        return result

In [28]:
user_count_list = {}
user_count_list['d'] = {}
user_count_list['c'] = {}
user_count_list['t'] = {}
user_count_list['g'] = {}

In [29]:
# get special part score
def user_info_score(user_key, inputs, to_id_dict, movie_info_dict, dict_name, is_cast=False):
    """
    inputs: list of tuples (movieID, rating) for one user
    output: list of tuples (InfoID, rating) for the user
    """
    result_list = []
    score_dict = {}
    for tup in inputs:
        rating = float(tup[1])
        info_list = movie_info_dict[str(tup[0])][0:3] if is_cast else movie_info_dict[str(tup[0])]
        info_list_prop = generate_proportion(info_list, is_cast)
        for one_info in info_list_prop:
            # one_info has form (info_id, proportion)
            info_id = to_id_dict[one_info[0]]
            if info_id not in score_dict:
                score_dict[info_id] = [(one_info[1], rating)]
            else:
                score_dict[info_id].append((one_info[1], rating))
    
    for info_key in score_dict:
        scores, proportion = core_fn(score_dict[info_key])
        result_list.append([user_key, info_key, scores])
        if user_key not in user_count_list[dict_name]:
            user_count_list[dict_name][user_key] = {}

        user_count_list[dict_name][user_key][info_key] = proportion
    
    return result_list

In [30]:
director_rating = []
num_of_1 = 0
for key in user_rating:
    director_rating += user_info_score(key, user_rating[key], director_2_id, director_dict, 'd')
    
print(num_of_1)
train_user_d = [int(info[0]) for info in director_rating]
train_movie_d = [int(info[1]) for info in director_rating]
train_rating_d = [float(info[2]) for info in director_rating]

pickle.dump(train_user_d, open('../data/'+FILE_NAME+'/train_user_d.pkl', 'wb'))
pickle.dump(train_movie_d, open('../data/'+FILE_NAME+'/train_movie_d.pkl', 'wb'))
pickle.dump(train_rating_d, open('../data/'+FILE_NAME+'/train_rating_d.pkl', 'wb'))

11230


In [31]:
num_of_1 = 0
genre_rating = []
for key in user_rating:
    genre_rating += user_info_score(key, user_rating[key], genre_2_id, genre_dict, 'g')
print(num_of_1)
train_user_g = [int(info[0]) for info in genre_rating]
train_movie_g = [int(info[1]) for info in genre_rating]
train_rating_g = [float(info[2]) for info in genre_rating]

pickle.dump(train_user_g, open('../data/'+FILE_NAME+'/train_user_g.pkl', 'wb'))
pickle.dump(train_movie_g, open('../data/'+FILE_NAME+'/train_movie_g.pkl', 'wb'))
pickle.dump(train_rating_g, open('../data/'+FILE_NAME+'/train_rating_g.pkl', 'wb'))

7475


In [32]:
num_of_1 = 0
cast_rating = []
for key in user_rating:
    cast_rating += user_info_score(key, user_rating[key], cast_2_id, cast_dict,'c', is_cast=True)

print(num_of_1)
train_user_c = [int(info[0]) for info in cast_rating]
train_movie_c = [int(info[1]) for info in cast_rating]
train_rating_c = [float(info[2]) for info in cast_rating]

pickle.dump(train_user_c, open('../data/'+FILE_NAME+'/train_user_c.pkl', 'wb'))
pickle.dump(train_movie_c, open('../data/'+FILE_NAME+'/train_movie_c.pkl', 'wb'))
pickle.dump(train_rating_c, open('../data/'+FILE_NAME+'/train_rating_c.pkl', 'wb'))

48630


In [33]:
topic_dict = pickle.load(open('../data/year2010_movie/topic_dict.pkl', 'rb'))

In [34]:
"""
def topic_core_fn(scores):
    # scores is the list of (proportion, rating)
    sum_p = 0.
    sum_r = 0.
    num = len(scores)
    for tup in scores:
        prop = float(tup[0]) 
        r = float(tup[1])
        sum_r += prop * r
        sum_p += prop
    
    return sum_r/sum_p * sigmoid(num)
"""


def user_topic_score(user_key, inputs):
    """
    inputs: list of tuples (movieID, rating) for one user
    output: list of tuples (castID, rating) for the user
    """
    result_list = []
    score_dict = {}
    for tup in inputs:
        rating = tup[1]
        # one_info has the form (type_id, proportion)

        for one_info in topic_dict[str(tup[0])]:
            type_id = one_info[0]
            prop = one_info[1]
            if type_id not in score_dict:
                score_dict[type_id] = [(prop, rating)]
            else:
                score_dict[type_id].append((prop, rating))

    
    for key in score_dict:
        scores, proportion = core_fn(score_dict[key])
        result_list.append([user_key, key, scores])
        if user_key not in user_count_list['t']:
            user_count_list['t'][user_key] = {}
        user_count_list['t'][user_key][key] = proportion
    
    return result_list

In [35]:
topic_rating = []
for key in user_rating:
    topic_rating += user_topic_score(key, user_rating[key])

train_user_t = [int(info[0]) for info in topic_rating]
train_movie_t = [int(info[1]) for info in topic_rating]
train_rating_t = [float(info[2]) for info in topic_rating]

pickle.dump(train_user_t, open('../data/'+FILE_NAME+'/train_user_t.pkl', 'wb'))
pickle.dump(train_movie_t, open('../data/'+FILE_NAME+'/train_movie_t.pkl', 'wb'))
pickle.dump(train_rating_t, open('../data/'+FILE_NAME+'/train_rating_t.pkl', 'wb'))

In [36]:
pickle.dump(user_count_list, open('../data/'+FILE_NAME+'/user_count_list.pkl', 'wb'))

In [37]:
pickle.dump(id_2_genre, open('../data/'+FILE_NAME+'/id_2_genre.pkl', 'wb'))
pickle.dump(id_2_director, open('../data/'+FILE_NAME+'/id_2_director.pkl', 'wb'))
pickle.dump(id_2_cast, open('../data/'+FILE_NAME+'/id_2_cast.pkl', 'wb'))

pickle.dump(genre_2_id, open('../data/'+FILE_NAME+'/genre_2_id.pkl', 'wb'))
pickle.dump(director_2_id, open('../data/'+FILE_NAME+'/director_2_id.pkl', 'wb'))
pickle.dump(cast_2_id, open('../data/'+FILE_NAME+'/cast_2_id.pkl', 'wb'))

In [38]:
pickle.dump(topic_rating, open('../data/'+FILE_NAME+'/topic_rating.pkl', 'wb'))
pickle.dump(director_rating, open('../data/'+FILE_NAME+'/director_rating.pkl', 'wb'))
pickle.dump(genre_rating, open('../data/'+FILE_NAME+'/genre_rating.pkl', 'wb'))
pickle.dump(cast_rating, open('../data/'+FILE_NAME+'/cast_rating.pkl', 'wb'))

In [39]:
#generate validation rating set
director_rating_v = []
for key in user_rating_validation:
    director_rating_v += user_info_score(key, user_rating_validation[key], director_2_id,  director_dict, 'd')

train_user_d_v = [int(info[0]) for info in director_rating_v]
train_movie_d_v = [int(info[1]) for info in director_rating_v]
train_rating_d_v = [float(info[2]) for info in director_rating_v]
    
cast_rating_v = []
for key in user_rating_validation:
    cast_rating_v += user_info_score(key, user_rating_validation[key], cast_2_id, cast_dict, 'c', is_cast=True)
    
train_user_c_v = [int(info[0]) for info in cast_rating_v]
train_movie_c_v = [int(info[1]) for info in cast_rating_v]
train_rating_c_v = [float(info[2]) for info in cast_rating_v]


genre_rating_v = []
for key in user_rating_validation:
    genre_rating_v += user_info_score(key, user_rating_validation[key], genre_2_id, genre_dict, 'g')  

train_user_g_v = [int(info[0]) for info in genre_rating_v]
train_movie_g_v = [int(info[1]) for info in genre_rating_v]
train_rating_g_v = [float(info[2]) for info in genre_rating_v]
    
topic_rating_v = []
for key in user_rating_validation:
    topic_rating_v += user_topic_score(key, user_rating_validation[key])

train_user_t_v = [int(info[0]) for info in topic_rating_v]
train_movie_t_v = [int(info[1]) for info in topic_rating_v]
train_rating_t_v = [float(info[2]) for info in topic_rating_v]    

pickle.dump(train_user_d_v, open('../data/'+FILE_NAME+'/train_user_d_v.pkl', 'wb'))
pickle.dump(train_movie_d_v, open('../data/'+FILE_NAME+'/train_movie_d_v.pkl', 'wb'))
pickle.dump(train_rating_d_v, open('../data/'+FILE_NAME+'/train_rating_d_v.pkl', 'wb'))

pickle.dump(train_user_g_v, open('../data/'+FILE_NAME+'/train_user_g_v.pkl', 'wb'))
pickle.dump(train_movie_g_v, open('../data/'+FILE_NAME+'/train_movie_g_v.pkl', 'wb'))
pickle.dump(train_rating_g_v, open('../data/'+FILE_NAME+'/train_rating_g_v.pkl', 'wb'))

pickle.dump(train_user_c_v, open('../data/'+FILE_NAME+'/train_user_c_v.pkl', 'wb'))
pickle.dump(train_movie_c_v, open('../data/'+FILE_NAME+'/train_movie_c_v.pkl', 'wb'))
pickle.dump(train_rating_c_v, open('../data/'+FILE_NAME+'/train_rating_c_v.pkl', 'wb'))

pickle.dump(train_user_t_v, open('../data/'+FILE_NAME+'/train_user_t_v.pkl', 'wb'))
pickle.dump(train_movie_t_v, open('../data/'+FILE_NAME+'/train_movie_t_v.pkl', 'wb'))
pickle.dump(train_rating_t_v, open('../data/'+FILE_NAME+'/train_rating_t_v.pkl', 'wb'))

In [40]:
pickle.dump(topic_rating_v, open('../data/'+FILE_NAME+'/topic_rating_v.pkl', 'wb'))
pickle.dump(director_rating_v, open('../data/'+FILE_NAME+'/director_rating_v.pkl', 'wb'))
pickle.dump(genre_rating_v, open('../data/'+FILE_NAME+'/genre_rating_v.pkl', 'wb'))
pickle.dump(cast_rating_v, open('../data/'+FILE_NAME+'/cast_rating_v.pkl', 'wb'))

In [45]:
#generate movie feature dict
#form as movieID: dict with key (director, type, genre, topic)
movie_feature_dict = {}

movie_id_set = set()
for key in movie_rating:
    movie_id_set.add(key)
for key in movie_rating_validation:
    movie_id_set.add(key)
for key in movie_rating_test:
    movie_id_set.add(key)

for m_id in movie_id_set:
    try:
        topic_list = [tup[0] for tup in topic_dict[str(m_id)]]
    except:
        topic_list = []

    director_list = [director_2_id[_id] for _id in director_dict[str(m_id)]]

    
    try:
        cast_list = [cast_2_id[_id] for _id in cast_dict[str(m_id)][0:3]]
    except:
        cast_list = []
    try:
        genre_list = [genre_2_id[_id] for _id in genre_dict[str(m_id)]]
    except:
        genre_list = []
    tmp_dict = {}
    tmp_dict['d'] = director_list
    tmp_dict['t'] = topic_list
    tmp_dict['g'] = genre_list
    tmp_dict['c'] = cast_list
    
    movie_feature_dict[m_id] = tmp_dict

In [42]:
pickle.dump(movie_feature_dict, open('../data/'+FILE_NAME+'/movie_feature_dict.pkl', 'wb'))

In [46]:
movie_feature_dict

{0: {'c': [0, 1, 2], 'd': [0], 'g': [0, 1], 't': [49]},
 1: {'c': [3, 4],
  'd': [1],
  'g': [2],
  't': [0,
   1,
   2,
   3,
   4,
   5,
   6,
   7,
   8,
   9,
   10,
   11,
   12,
   13,
   14,
   15,
   16,
   17,
   18,
   19,
   20,
   21,
   22,
   23,
   24,
   25,
   26,
   27,
   28,
   29,
   30,
   31,
   32,
   33,
   34,
   35,
   36,
   37,
   38,
   39,
   40,
   41,
   42,
   43,
   44,
   45,
   46,
   47,
   48,
   49]},
 3: {'c': [6, 7, 8], 'd': [3], 'g': [4, 6, 7], 't': [1, 46]},
 4: {'c': [9, 10, 11], 'd': [4], 'g': [8, 2, 9, 10, 11], 't': [2, 26, 32, 34]},
 5: {'c': [12, 13, 14], 'd': [5], 'g': [8, 12, 6], 't': [17]},
 6: {'c': [15, 16, 17], 'd': [6], 'g': [10], 't': [2, 3, 34]},
 7: {'c': [18, 19, 20], 'd': [7], 'g': [8, 10, 11, 13], 't': [2, 34]},
 8: {'c': [21, 22, 23], 'd': [8], 'g': [2, 4], 't': [28, 32, 37]},
 9: {'c': [24, 25, 26], 'd': [9], 'g': [14, 10, 11], 't': [3, 46]},
 10: {'c': [27, 28, 29], 'd': [10], 'g': [3], 't': [14, 49]},
 11: {'c': [30, 31,

In [47]:
len(movie_feature_dict)

3643